In [ ]:
import streamlit as st
import openai
from dotenv import load_dotenv
import time
from datetime import datetime
import trafilatura as tf


load_dotenv()

# OpenAI API 설정
client = openai.OpenAI()
assistant_id = "asst_WBJHxUaCHCBheZWNA1yYMJUf"
thread_id = "thread_U3wPxZxDgg0wnJTRFkyLWiTz"

# 여행 계획 생성 함수
def generate_travel_plan(destination, duration, budget, blog_url):
    try:
        # 사용자 입력 정보 전송
        user_input = f"여행 지역: {destination}, 여행 기간: {duration} 일, 예산: {budget} 원, 참고 웹 페이지: {blog_url}"
        client.beta.threads.messages.create(thread_id=thread_id, role="user", content=user_input)
        
        # 여행 계획 생성 요청
        run = client.beta.threads.runs.create(assistant_id=assistant_id, thread_id=thread_id)
        run_id = run.id
        
        # 실행 완료 대기
        while True:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if run.completed_at:
                # 실행 완료 시간 계산
                elapsed_time = run.completed_at - run.created_at
                formatted_elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
                st.success(f"여행 계획 생성 완료 ({formatted_elapsed_time})")
                
                # 최종 메시지 확인
                messages = client.beta.threads.messages.list(thread_id=thread_id)
                last_message = messages.data[0]
                travel_plan = last_message.content[0].text.value
                return travel_plan
            
            time.sleep(5)  # 5초마다 확인
            
    except Exception as e:
        st.error(f"An error occurred: {e}")
        return None
    
# Streamlit 애플리케이션 시작
def main():
    # 페이지 설정
    st.set_page_config(
        page_title="Travel Planner Assistant",
        page_icon="🌍",
        layout="centered",
        initial_sidebar_state="expanded",
    )
    
    # 사이드바
    with st.sidebar:
        st.title("Travel Planner Assistant")
        st.image("https://pbs.twimg.com/media/E-LYlnoVIAMxx0P.jpg", use_column_width=True)
        st.markdown("""
        ### 기능 설명
        - 원하는 여행지와 기간, 예산에 맞춘 여행 계획 짜기
        - 항공권이나 기차 예매 방법 안내
        """)
    
    # 메인 페이지
    st.title("🌍 Travel Planner Assistant")
    st.subheader("여행 계획을 세워보세요!")

    # 사용자 입력 받기
    destination = st.text_input("여행 지역")
    duration = st.number_input("여행 기간(일)", min_value=1, step=1)
    budget = st.number_input("여행 예산(원)", min_value=1, step=1)
    blog_url = st.text_input("참고할 웹 페이지 URL")
    
    html = tf.fetch_url(blog_url)
    result = tf.extract(html)
    
    if st.button("계획 짜기"):
        if destination and duration and budget and blog_url:
            with st.spinner("여행 계획을 세우고 있습니다..."):
                travel_plan = generate_travel_plan(destination, duration, budget, result)
                if travel_plan:
                    st.markdown(f"### 여행 계획:\n{travel_plan}")
        else:
            st.warning("모든 정보를 입력해주세요.")

if __name__ == "__main__":
    main()